[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/boostcourse-ds-511/blob/master/pima-classification-baseline-07.ipynb)


## 데이터셋 출처
* [Pima Indians Diabetes Database | Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database)
* https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html


### 데이터 구성

* Pregnancies : 임신 횟수
* Glucose : 2시간 동안의 경구 포도당 내성 검사에서 혈장 포도당 농도 
* BloodPressure : 이완기 혈압 (mm Hg)
* SkinThickness : 삼두근 피부 주름 두께 (mm), 체지방을 추정하는데 사용되는 값
* Insulin : 2시간 혈청 인슐린 (mu U / ml)
* BMI : 체질량 지수 (체중kg / 키(m)^2)
* DiabetesPedigreeFunction : 당뇨병 혈통 기능
* Age : 나이
* Outcome : 768개 중에 268개의 결과 클래스 변수(0 또는 1)는 1이고 나머지는 0입니다.


## 필요한 라이브러리 로드

In [1]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy
# 시각화를 위한 seaborn, matplotlib.pyplot 을 로드합니다. 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## 데이터셋 로드

In [4]:
df = pd.read_csv("data/diabetes_feature.csv")
df.shape

(768, 16)

In [5]:
df_insulin = pd.read_csv("data/diabetes_fill_insulin.csv")
df["Insulin"] = df_insulin["Insulin"]
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,184.855904,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,66.159991,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,194.151175,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94.000000,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.000000,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


In [6]:
# 데이터셋을 미리보기 합니다.

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,184.855904,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,66.159991,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,194.151175,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94.000000,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.000000,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


## 학습과 예측에 사용할 데이터셋 만들기

In [7]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [8]:
X = df[['Glucose', 'BloodPressure', 'SkinThickness',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high',
       'Insulin']]
X.shape

(768, 8)

In [9]:
y = df['Outcome']
y.shape

(768,)

In [10]:
# 사이킷런에서 제공하는 model_selection 의 train_test_split 으로 만듭니다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [11]:
# train 세트의 문제와 정답의 데이터 수를 확인해 주세요.

X_train.shape, y_train.shape

((614, 8), (614,))

In [12]:
# test 세트의 문제와 정답의 데이터 수를 확인해 주세요.

X_test.shape, y_test.shape

((154, 8), (154,))

## 여러 개의 알고리즘을 사용해서 비교하기

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
             RandomForestClassifier(random_state=42),
             GradientBoostingClassifier(random_state=42)
            ]
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [14]:
max_depth = np.random.randint(2, 20, 10)
max_depth

array([ 8, 17, 17,  8, 15,  9, 12, 15,  2, 17])

In [15]:
max_features = np.random.uniform(0.3, 1.0, 10)
max_features

array([0.41952291, 0.39081305, 0.50452963, 0.39216829, 0.6897467 ,
       0.66072302, 0.44382436, 0.84044139, 0.30976013, 0.42879202])

In [16]:
results = []
for estimator in estimators:
    result = []
    result.append(estimator.__class__.__name__)
    results.append(result)
results

[['DecisionTreeClassifier'],
 ['RandomForestClassifier'],
 ['GradientBoostingClassifier']]

In [17]:
# param_distributions["n_estimators"] = np.random.randint(100, 1000, 10)
# param_distributions

In [19]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

param_distributions = {"max_depth": max_depth, 
                       "max_features": max_features}

results = []
for estimator in estimators:
    result = []
    if estimator.__class__.__name__ != 'DecisionTreeClassifier':
        param_distributions["n_estimators"] = np.random.randint(100, 200, 10)
        
    clf = RandomizedSearchCV(estimator, 
                       param_distributions, 
                       n_iter=10,
                       scoring="accuracy",
                       n_jobs=-1,
                       cv=5, 
                       verbose=2
                      )

    clf.fit(X_train, y_train)
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(X_test, y_test))
    result.append(clf.cv_results_)
    results.append(result)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [20]:
df = pd.DataFrame(results, 
             columns=["estimator", "best_params", "train_score", "test_score", "cv_result"])
df

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.8992628262122362, 'max_dept...",0.775330,0.701299,"{'mean_fit_time': [0.011397314071655274, 0.012..."
1,RandomForestClassifier,"{'n_estimators': 170, 'max_features': 0.902251...",0.791603,0.720779,"{'mean_fit_time': [0.4853985786437988, 0.83199..."
2,GradientBoostingClassifier,"{'n_estimators': 166, 'max_features': 0.357008...",0.786712,0.707792,"{'mean_fit_time': [1.8991844654083252, 2.43075..."


In [21]:
pd.DataFrame(df.loc[1, "cv_result"]).sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,0.721596,0.050335,0.041001,0.001414,170,0.902251,4,"{'n_estimators': 170, 'max_features': 0.902251...",0.813008,0.837398,0.731707,0.739837,0.836066,0.791603,0.046475,1
9,0.775000,0.134159,0.050001,0.010898,152,0.414821,14,"{'n_estimators': 152, 'max_features': 0.414820...",0.804878,0.837398,0.747967,0.747967,0.819672,0.791577,0.037066,2
1,0.831997,0.050644,0.054205,0.025675,143,0.899263,14,"{'n_estimators': 143, 'max_features': 0.899262...",0.804878,0.845528,0.739837,0.756098,0.811475,0.791563,0.038521,3
8,0.844572,0.028578,0.046013,0.006418,152,0.880748,19,"{'n_estimators': 152, 'max_features': 0.880748...",0.813008,0.837398,0.731707,0.764228,0.811475,0.791563,0.038176,4
5,0.706413,0.110789,0.087256,0.043476,145,0.414821,17,"{'n_estimators': 145, 'max_features': 0.414820...",0.796748,0.829268,0.747967,0.747967,0.819672,0.788325,0.034605,5
0,0.485399,0.012468,0.041402,0.005003,106,0.414821,17,"{'n_estimators': 106, 'max_features': 0.414820...",0.788618,0.821138,0.756098,0.756098,0.811475,0.786685,0.027116,6
6,0.871152,0.150819,0.061599,0.048966,106,0.463074,17,"{'n_estimators': 106, 'max_features': 0.463073...",0.788618,0.821138,0.756098,0.756098,0.811475,0.786685,0.027116,6
3,1.030400,0.128305,0.054801,0.014580,153,0.728756,8,"{'n_estimators': 153, 'max_features': 0.728755...",0.772358,0.837398,0.739837,0.731707,0.811475,0.778555,0.040690,8
7,0.567931,0.104131,0.043196,0.018979,108,0.357009,13,"{'n_estimators': 108, 'max_features': 0.357008...",0.788618,0.821138,0.731707,0.731707,0.803279,0.775290,0.037046,9
2,0.717204,0.059690,0.050198,0.006369,153,0.899263,3,"{'n_estimators': 153, 'max_features': 0.899262...",0.772358,0.788618,0.747967,0.723577,0.803279,0.767160,0.028495,10
